In [1]:
# importiere Library und build Instanz

from camera_roboter import CameraRoboter
from math import pi
import rospy

CameraRoboterI = CameraRoboter(load_camera=True)

[ INFO] [1685980528.732068040]: Loading robot model 'panda'...
[ INFO] [1685980530.101374201]: Ready to take commands for planning group panda_arm.
Initialize workspace...
============ Planning frame: panda_link0
============ End effector link: panda_link8
============ Available Planning Groups: ['panda_arm']
============ Printing robot state
joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "panda_link0"
  name: 
    - panda_joint1
    - panda_joint2
    - panda_joint3
    - panda_joint4
    - panda_joint5
    - panda_joint6
    - panda_joint7
  position: [-8.429131939447599e-05, -0.785701085838898, 8.028513330183266e-05, -2.35589592835127, -0.0003245282839577271, 1.57125472974228, -0.7851913144370203]
  velocity: []
  effort: []
multi_dof_joint_state: 
  header: 
    seq: 0
    stamp: 
      secs: 0
      nsecs:         0
    frame_id: "panda_link0"
  joint_names: []
  transforms: []
  twist: []
  wrench: []
attached_collision_objects:

In [2]:
# prüfe scharfes bild und Position

try:
    CameraRoboterI.Camera.init_camera()
    CameraRoboterI.move_ready_pose()
    CameraRoboterI.get_video_stream()
finally:
    CameraRoboterI.Camera.close()
    CameraRoboterI.return_to_ready_pose()

# labeln jeder Stelle der Schweißnah

try:
    CameraRoboterI.Camera.init_camera()
    CameraRoboterI.calibration(0.016, 5)
finally:
    CameraRoboterI.Camera.close()
    CameraRoboterI.return_to_ready_pose()

initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
StopLiveVideo returns 0
ExitCamera returns 0
initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
StopLiveVideo returns 0
ExitCamera returns 0


In [3]:
# eingebe Info der Probe

dict_info = dict()

dict_info['id'] = "5_004"
dict_info['speed'] = "500"
dict_info['rotate_speed'] = "2100"
dict_info['date'] = "06.11.19"

training_label = list(range(6))
training_label[0] = 1
training_label[1] = 1
training_label[2] = 1
training_label[3] = 1
training_label[4] = 1
training_label[5] = 1

dict_info['training_label'] = training_label
CameraRoboterI.DataManager.set_basic_info(dict_info)
dict_info

{'id': '5_004',
 'speed': '500',
 'rotate_speed': '2100',
 'date': '06.11.19',
 'training_label': [1, 1, 1, 1, 1, 1]}

In [4]:
# Hauptprogramm

try:
    # initialize Kamera
    CameraRoboterI.Camera.init_camera()

    # setzt Heilligkeitstufe
    brightness_class = [30, 50, 70, 100]

    # jeder Heilligkeitstufe kommplette Prozess laufen
    for i in range(1): #len(brightness_class)

        # ---linear Bewegung---
        CameraRoboterI.DataManager.set_brightness(brightness_class[i])
        CameraRoboterI.move_pose(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init, CameraRoboterI.roll_angle_init, CameraRoboterI.pitch_angle_init, CameraRoboterI.yaw_angel_init, save_image=True)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)
        CameraRoboterI.return_to_ready_pose()

        # ---drehe um z-Achse---
        ## kleine Winkel von pi/100
        angle = CameraRoboterI.DataManager.set_rotate_angle_z(pi/100)
        CameraRoboterI.rotate(angle)
        CameraRoboterI.move_position(CameraRoboterI.x_init, CameraRoboterI.y_init-0.001, CameraRoboterI.z_init, save_image=True)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)
        CameraRoboterI.return_to_ready_pose()

        ## etwa größe Winkel von pi/50
        angle = CameraRoboterI.DataManager.set_rotate_angle_z(pi/50)
        CameraRoboterI.rotate(angle)
        CameraRoboterI.move_position(CameraRoboterI.x_init, CameraRoboterI.y_init-0.002, CameraRoboterI.z_init, save_image=True)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)
        CameraRoboterI.return_to_ready_pose()

        # setzt Winkel wieder zurück auf Null
        _ = CameraRoboterI.DataManager.set_rotate_angle_z(0)

        # ---drehe um x-Achse--- um pi/25
        CameraRoboterI.move_position(CameraRoboterI.x_init, CameraRoboterI.y_init+0.0035, CameraRoboterI.z_init)
        angle = CameraRoboterI.DataManager.set_rotate_angle_x(pi/25)
        CameraRoboterI.change_angle(angle)
        CameraRoboterI.get_image(take_image=True, image_index=0)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)
        CameraRoboterI.return_to_ready_pose()

        # setzt Winkel wieder zuruck auf Null
        _ = CameraRoboterI.DataManager.set_rotate_angle_x(0)

        # ---unscharfe manuell einstellen---
        ## manuell einstellen
        CameraRoboterI.move_position(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init)
        CameraRoboterI.get_video_stream()

        ## linear Bewegung
        CameraRoboterI.DataManager.set_zoom_blur(1)
        CameraRoboterI.move_position(CameraRoboterI.x_init, CameraRoboterI.y_init, CameraRoboterI.z_init, save_image=True)
        CameraRoboterI.move_linear_trajectory(0.016, 5, save_image=True)
        CameraRoboterI.return_to_ready_pose()

        # setzt blur wieder zurück
        CameraRoboterI.DataManager.set_zoom_blur(0)

        # ---manuell Heilligkeit einstellen---
        _ = input("Bitte stellt Heilligkeit manuell auf {} ein!".format(brightness_class[i]))

finally:
    # schließe Kamera und zurück zur Initialposition
    CameraRoboterI.Camera.close()
    CameraRoboterI.return_to_ready_pose()




initCamera returns 0
SetColorMode IS_CM_BGR8_PACKED returns 0
setExposure returns 0
AOI IS_AOI_IMAGE_SET_AOI returns 0
AllocImageMem returns 0
SetImageMem returns 0
CaptureVideo returns 0
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:47_0.43_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:48_0.45_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:49_0.46_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:50_0.48_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:52_0.49_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:53_0.51_30_0_0_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:58_0.43_30_0_0.03_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:22:59_0.45_30_0_0.03_0_1.jpg
image is saved als name: images/5_004_500_2100/05.06.2023_17:23:00_0.46_30_0_0.03_0_1.jpg
image is saved als n